```json
            { 
                "tag": "q_0000_0001", 
                "type": "question", 
                "description": { 
                    "number": "", 
                    "question": "", 
                    "options": [], 
                    "options": null, 
                    "answer": "", 
                    "explanation": "" 
                    }, 
                "caption": [ 
                    " 기출", "키워드: " 
                    ], 
                "file_path": null, 
                "bbox": null 
            }, 
{ 
    "tag": "note_0000_0001", 
    "type": "footnote", 
    "description": "1 이러한 계약을 법적으로는 금전소비대차계약이라고 한다. 차용증서를 영어로는 I owe you.의 소리를 따라 IOU 라고 한다.", 
    "caption": null, 
    "file_path": null, 
    "bbox": null 
}, 
{ 
    "tag": "tb_0000_0001", 
    "type": "table", 
    "description": "", 
    "caption": [], 
    "file_path": "??????/crop/tb_0000_0001.png", 
    "bbox": null 
}, 
{ 
    "tag": "img_0000_0001", 
    "type": "image", 
    "description": null, 
    "caption": [], 
    "file_path": "?????/crop/img_0000_0001.png", 
    "bbox": null 
}
```

### 도서 정보 추출

In [4]:
import tools.ProcessFiles as pre_b
excel = pre_b.get_excel_data(1) # i차 분석

In [5]:
import pandas as pd
import os

base_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
analysis = {1:'1차 분석', 2:'2차 분석', 3: '3차 분석'}
buy = {1:'1차 구매', 2:'2차 구매', 3: '3차 구매'}
i = 1

excel_analy = pd.read_excel(os.path.join(base_path, '도서목록_전체통합.xlsx'), sheet_name=analysis[i], header=3)[['관리번호_new','관리번호', 'ISBN', '도서명', '분류']]
# excel_buy = pd.read_excel(os.path.join(base_path, '도서목록_전체통합.xlsx'), sheet_name=buy[i], header=4)[['ISBN', '도서명', '출판일', '코퍼스 1분류', '코퍼스 2분류', '비고']]
# excel_buy.fillna("", inplace=True)

# merge_excel = pd.merge(excel_analy, excel_buy, on=['ISBN', '도서명'], how='inner')
# merge_excel = merge_excel[['관리번호', 'ISBN', '도서명', '출판일', '코퍼스 1분류', '코퍼스 2분류', '비고', '분류']].set_index('관리번호')

In [24]:
excel_analy

,관리번호_new,관리번호,ISBN,도서명,분류
0,SS0319,225933255,9791163478782,금융의 현재와 미래,Lv3/4
1,SS0320,239373741,9788984059900,요즘 금리 쉬운 경제,Lv3/4
2,SS0321,243228629,9788960542570,알기 쉬운 보험 세테크 100% 활용법,Lv3/4
3,SS0322,248779244,9788957614358,공정거래 용어사전,Lv2
4,SS0323,250217022,9791190820660,1페이지 보험 정리의 기술,Lv3/4
...,...,...,...,...,...
101,SS0420,361239919,9791138370714,2025 시대에듀 차량손해사정사 2차 한권으로 끝내기,Lv3/4
102,SS0421,361988234,9791138392440,新2025 시대에듀 hoa 재경관리사 전과목 핵심이론+적중문제+기출 동형문제 한권으...,Lv5
103,SS0422,362874088,9791138392938,2025~2026 시대에듀 PASSCODE 증권투자권유자문인력 실제유형 모의고사 4...,Lv5
104,SS0423,364690338,9791138394239,2025~2026 시대에듀 펀드투자권유자문인력 한권으로 끝내기,Lv5


### Lv2 파일명 변경, Lv3/4/5 폴더 분류

In [ ]:
import tools.ProcessFiles as pf
import shutil
# pre_b.move_jsons(1) # i차 분석

# pf.change_names(excel_analy, '관리번호', '관리번호_new', '')
# file_path = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL/1C/Lv2/248779244/248779244.json'

for fn, tn, lv in zip(excel_analy['관리번호'], excel_analy['관리번호_new'], excel_analy['분류']):
    fn = str(fn)
    if '/' in lv: 
       lv = lv.replace("/", '_')
#     print(fn, tn, lv)
    
    dir = os.path.join(pf.FINAL_DATA_PATH, pf.CYCLE_PATH[1], lv)
    os.system(f'cd {dir} && mv {fn}_low {tn}_low && cd {tn} &&'+f' for file in *{fn}*; do mv "$file" "$'+'{file//'+fn+'/'+tn+'}"; done')
    print(os.path.join(dir, fn))
    # break


---

### 개별 처리
- 불필요 페이지 제거
- 오타
- 페이지 머리말/꼬리말

In [3]:
import tools.ProcessWorkbook.quizbook3 as quiz

# quiz.main(json_files[0])

## 전체 돌리기

In [25]:
import os, json
from typing import List, Dict, Any
import tools.ProcessFiles as pf
import tools.ProcessLv2 as pl2

json_files = pf.get_filelist(1)
json_files

['/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/1C/Lv2/SS0322/SS0322.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/1C/Lv2/SS0329/SS0329.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/1C/Lv2/SS0330/SS0330.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/1C/Lv2/SS0338/SS0338.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/1C/Lv2/SS0339/SS0339.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/1C/Lv2/SS0342/SS0342.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/1C/Lv2/SS0345/SS0345.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/1C/Lv2/SS0351/SS0351.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/1C/Lv2/SS0352/SS0352.json',
 '/Users/jinym/Libr

In [19]:
# excel = pf.get_excel_data(1)

In [26]:
# excel

In [ ]:
os.path.splitext(os.path.basename(json_files[0]))[0]

In [3]:
import json
import tools.ProcessLv2 as pl2

# for j in json_files:
if True:
    j = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL/3C_0902/Lv2/SS0275/SS0275.json'
    INPUT_PATH = j
    BACKUP_PATH = INPUT_PATH + ".bak"

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        origin = json.load(f)

    with open(BACKUP_PATH, "w", encoding="utf-8") as f:
        json.dump(origin, f, ensure_ascii=False, indent=4)

    # id = excel[excel['ISBN'] == int(origin.get('file_id'))].index[0]
    # new = {
    #     'file_id': os.path.splitext(os.path.basename(j))[0],
    #     'ISBN': str(excel.loc[id, 'ISBN']),
    #     'title': excel.loc[id, '도서명'],
    #     'cat1_domain': excel.loc[id, '코퍼스 1분류'],
    #     'cat2_sub': excel.loc[id, '코퍼스 2분류'],
    #     'cat3_specific': excel.loc[id, '비고'],
    #     'pub_date': str(excel.loc[id, '출판일'])[:10],
    #     # 'contents': [],
    #     'contents': origin['contents']
    # }
    
    # new = pl2.fill_chapter(origin)
    # new = pl2.merge_paragraphs(origin)
    new = origin

    # for i in range(len(new['contents'])):
        # contents = new['contents'][i]
        # # c = extract_qna(contents)
        # page_contents = pl2.remove_enter(contents['page_contents'])
        # contents['page_contents'] = page_contents
        # # c = contents

    with open(INPUT_PATH.replace(".json", ".json"), "w", encoding="utf-8") as f:
        json.dump(new, f, ensure_ascii=False, indent=4)

In [ ]:
# for js in json_files:
#     INPUT_PATH = js
#     BACKUP_PATH = INPUT_PATH + ".bak"

#     if not os.path.exists(INPUT_PATH):
#         raise FileNotFoundError(INPUT_PATH)

#     with open(INPUT_PATH, "r", encoding="utf-8") as f:
#         data = json.load(f)

#     with open(BACKUP_PATH, "w", encoding="utf-8") as f:
#         json.dump(data, f, ensure_ascii=False, indent=4)

#     contents: List[Dict[str, Any]] = data.get("contents", [])
#     data["contents"] = build_output(contents)

#     with open(INPUT_PATH, "w", encoding="utf-8") as f:
#         json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
for i, j in enumerate(json_files):
    if '276709608' in j:
        print(i, j)